In [2]:
"""
Объединение данных из S3 и Postgres для анализа активных клиентов
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

def main():
    # Конфигурация
    ORDERS_S3_PATH = "s3a://startde-project/roma_mesh/orders_filtered"
    PG_HOST = "my-postgres-host"
    PG_DATABASE = "my-database"
    PG_USER = "my-username"
    PG_PASSWORD = "my-password"

    # Инициализация Spark
    spark = SparkSession.builder \
        .appName("ActiveCustomersAnalysis") \
        .config("spark.hadoop.fs.s3a.access.key", "${S3_ACCESS_KEY}") \
        .config("spark.hadoop.fs.s3a.secret.key", "${S3_SECRET_KEY}") \
        .getOrCreate()

    # 1. Чтение данных из S3
    orders_df = spark.read.parquet(ORDERS_S3_PATH)
    print(f"Загружено заказов из S3: {orders_df.count()}")

    # 2. Чтение данных из Postgres
    postgres_url = f"jdbc:postgresql://{PG_HOST}:5432/{PG_DATABASE}"
    connection_properties = {
        "user": PG_USER,
        "password": PG_PASSWORD,
        "driver": "org.postgresql.Driver"
    }
    
    active_customers_df = spark.read.jdbc(
        url=postgres_url, 
        table="orders_filtered", 
        properties=connection_properties
    )
    print(f"Загружено активных клиентов: {active_customers_df.count()}")

    # 3. JOIN операция
    joined_df = orders_df.join(
        active_customers_df, 
        orders_df.customer_id == active_customers_df.order_customer_id
    )
    print(f"Результат JOIN: {joined_df.count()} записей")

    # 4. Подготовка результата
    result_df = joined_df.select(
        col("order_id"),
        to_date(col("order_purchase_timestamp")).alias("order_date"),
        col("order_customer_id"),
        col("order_status"),
        col("order_count")
    )

    # 5. Запись в Postgres
    result_df.write \
        .mode("overwrite") \
        .jdbc(url=postgres_url, table="task_4_result_table", properties=connection_properties)
    
    print("Данные записаны в таблицу task_4_result_table")
    
    spark.stop()

if __name__ == "__main__":
    main()


ФИНАЛЬНЫЙ ОТЧЕТ
